In [1]:
import pandas as pd
from tqdm import tqdm
import os
import numpy as np
from pathlib import Path
import csv
import random

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
# import lightgbm as lgb

from sklearn.metrics import (precision_score, recall_score, roc_auc_score, accuracy_score, mean_squared_error,
                             confusion_matrix, precision_recall_curve, roc_curve, brier_score_loss)

from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

In [3]:
# Random Forest Module -> hyperparameter tuning
def RF_pred_tuned(train_path, validation_path, train_column_drop, validation_column, details=None):
    
    df_train = pd.read_csv(train_path)
    df_validation = pd.read_csv(validation_path)
    
    X = df_train.drop(train_column_drop, axis = 1)
    y = df_train[validation_column]
    
    X_validation = df_validation.drop(train_column_drop, axis = 1)
    y_validation = df_validation[validation_column]
    
    # Build models with hyperparameters sets
    RSC = RandomizedSearchCV(
        estimator=RandomForestRegressor(),
        param_distributions={
            'n_estimators': range(1, 200, 10),
            'max_depth': range(1, 100, 10),
            'max_features': ['auto', 'sqrt', 'log2']}, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
    
    # Fit RandomizedSearchCV to find best hyperparameters
    search_result = RSC.fit(X, y.values.ravel())
    print("Best using: ", search_result.best_params_, "Score: ", search_result.best_score_)

    # Build models with optimized hyperparameters
    model_RF = RandomForestRegressor(
        n_estimators=search_result.best_params_["n_estimators"],
        max_depth=search_result.best_params_["max_depth"],
        max_features=search_result.best_params_["max_features"]
    )
    
    model_RF.fit(X, y.values.ravel())
    train_pred = model_RF.predict(X)
    y_pred = model_RF.predict(X_validation)
    
    train_rmse = mean_squared_error(y, train_pred, squared=False)
    
    test_rmse = mean_squared_error(y_validation, y_pred, squared=False)

    details['training_rmse'] = train_rmse
    details['testing_rmse'] = test_rmse

    print('Training RMSE: ', train_rmse, 'Testing RMSE: ', test_rmse)
    
    return model_RF

In [2]:
# Random Forest Module -> default parameters
def RF_pred_default(train_path, validation_path, train_column_drop, validation_column, details=None):
    
    df_train = pd.read_csv(train_path)
    df_validation = pd.read_csv(validation_path)
    
    X = df_train.drop(train_column_drop, axis = 1)
    y = df_train[validation_column]
    
    X_validation = df_validation.drop(train_column_drop, axis = 1)
    y_validation = df_validation[validation_column]
    
    model_RF = RandomForestRegressor()
    
    model_RF.fit(X, y.values.ravel())
    train_pred = model_RF.predict(X)
    y_pred = model_RF.predict(X_validation)
    
    train_rmse = mean_squared_error(y, train_pred, squared=False)
    
    test_rmse = mean_squared_error(y_validation, y_pred, squared=False)

    details['training_rmse'] = train_rmse
    details['testing_rmse'] = test_rmse

    print('Training RMSE: ', train_rmse, 'Testing RMSE: ', test_rmse)
    
    return model_RF

In [7]:
# dataset metadata

data_dir = "/home/asim/ssriva59/setup-stuff/gateway_and_dataset"
datasets = [
    "Facebook_data", # -> done,
    "Features_TestSet", # -> done,
    "House_Price_Adv_Regression", # -> done,
    # # "Instant_Liking",
    "Insurance", # -> done,
    "Isolet",
    "new_data_trans", # -> done,
    "OnlineNewsPopularity", # -> done,
    "ParkinsonData", # -> done,
    "Sberbank_Russian_Housing_Market", # -> done, Gradiant Boosting remains
    "slice_localization_data", # -> done,
    "Telecom_data", # -> done,
    "yearMSD_new", # -> done,
    "arrhythmia", # -> done,
    "Big_mart_sales", # -> done,
    "blogData", # -> done
    "communities", # -> done,
    "dengue_features", # -> done,
    "ECG0_p02", # -> done,
    "ENERGY_DATA_COMPLETE", # -> done
]

x_column_drop = {
    "Facebook_data": ['Unnamed: 0', 'Total.Interactions'],
    "Features_TestSet": ['Unnamed: 0', 'Target'],
    "House_Price_Adv_Regression": ['Unnamed: 0', 'SalePrice'],
    # "Instant_Liking": ['Instant.Liking'] -> NaN needs to be fixed,
    "Insurance": ['Unnamed: 0', 'charges'],
    "Isolet": ['Unnamed: 0', 'Target'], # -> some random error, need to see,
    "new_data_trans": ['Unnamed: 0', 'X23.Humedad_Exterior_Sensor'],
    "OnlineNewsPopularity": ['Unnamed: 0', 'shares'],
    "ParkinsonData": ['Unnamed: 0', 'total_UPDRS'],
    "Sberbank_Russian_Housing_Market": ['Unnamed: 0', 'price_doc'],
    "slice_localization_data": ['Unnamed: 0', 'reference'],
    "Telecom_data": ['Unnamed: 0', 'Churned.Label'],
    "yearMSD_new": ['Unnamed: 0', 'Year'],
    "arrhythmia": ['Unnamed: 0', 'Defection'],
    "Big_mart_sales": ['Unnamed: 0', 'Item_Outlet_Sales'],
    "blogData": ['Unnamed: 0', 'Comments'],
    "communities": ['Unnamed: 0','ViolentCrimesPerPop'],
    "dengue_features": ['Unnamed: 0','total_cases'],
    "ECG0_p02": ['Unnamed: 0', 'CurrentValue'],
    "ENERGY_DATA_COMPLETE": ['Unnamed: 0', 'Appliances']
}

target_column = {
    "Facebook_data": ['Total.Interactions'],
    "Features_TestSet": ['Target'],
    "House_Price_Adv_Regression": ['SalePrice'],
    # "Instant_Liking": ['Instant.Liking'] -> NaN needs to be fixed,
    "Insurance": ['charges'],
    "Isolet": ['Target'], # -> some random error, need to see,
    "new_data_trans": ['X23.Humedad_Exterior_Sensor'],
    "OnlineNewsPopularity": ['shares'],
    "ParkinsonData": ['total_UPDRS'],
    "Sberbank_Russian_Housing_Market": ['price_doc'],
    "slice_localization_data": ['reference'],
    "Telecom_data": ['Churned.Label'],
    "yearMSD_new": ['Year'],
    "arrhythmia": ['Defection'],
    "Big_mart_sales": ['Item_Outlet_Sales'],
    "blogData": ['Comments'],
    "communities": ['ViolentCrimesPerPop'],
    "dengue_features": ['total_cases'],
    "ECG0_p02": ['CurrentValue'],
    "ENERGY_DATA_COMPLETE": ['Appliances']
}

seeds = [1, 50, 100, 150, 200, 250, 300, 350, 400, 450]

In [ ]:
# Driver for tuned RF

for index, dataset in enumerate(datasets):
    df = pd.DataFrame()
    print(dataset)
    for seed in tqdm(seeds):
        csv_path_train = os.path.join(data_dir, dataset, "Train", dataset + '_seed_' + str(seed) + '_train.csv')
        csv_path_validation = os.path.join(data_dir, dataset, "Train", dataset + '_seed_' + str(seed) + '_validation.csv')
        csv_path_test = os.path.join(data_dir, dataset, "Test", dataset + '_Test_seed' + str(seed) + '_modified.csv')
        df_test = pd.read_csv(csv_path_test)
        
        X_test = df_test.drop(x_column_drop[dataset], axis = 1)
        y_test = df_test[target_column[dataset]]
        
        details = {
            'dataset': dataset,
            'seed': str(seed)
        }
        
        # For RF
        model_RF = RF_pred_tuned(csv_path_train, csv_path_validation, x_column_drop[dataset], target_column[dataset], details)
        
        importances = model_RF.feature_importances_
        indices = np.argsort(importances)[::-1]
        top_k = 10
        top_indices = indices[:top_k]
        details['best_feature_list'] = np.array(X_test.columns)[indices][0:top_k]
        
        y_pred = model_RF.predict(X_test)
        validation_rmse = mean_squared_error(y_test, y_pred, squared=False)
        details['validation_rmse'] = validation_rmse
        
        df = df.append(details, ignore_index=True)
        filepath = Path(dataset + '_RF_tuned.csv')
        filepath.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(filepath, index=False)

In [6]:
# Driver for default RF

for index, dataset in enumerate(datasets):
    df = pd.DataFrame()
    print(dataset)
    for seed in tqdm(seeds):
        csv_path_train = os.path.join(data_dir, dataset, "Train", dataset + '_seed_' + str(seed) + '_train.csv')
        csv_path_validation = os.path.join(data_dir, dataset, "Train", dataset + '_seed_' + str(seed) + '_validation.csv')
        csv_path_test = os.path.join(data_dir, dataset, "Test", dataset + '_Test_seed' + str(seed) + '_modified.csv')
        df_test = pd.read_csv(csv_path_test)
        
        X_test = df_test.drop(x_column_drop[dataset], axis = 1)
        y_test = df_test[target_column[dataset]]
        
        details = {
            'dataset': dataset,
            'seed': str(seed)
        }
        
        # For RF
        model_RF = RF_pred_default(csv_path_train, csv_path_validation, x_column_drop[dataset], target_column[dataset], details)
        
        importances = model_RF.feature_importances_
        indices = np.argsort(importances)[::-1]
        top_k = 10
        top_indices = indices[:top_k]
        details['best_feature_list'] = np.array(X_test.columns)[indices][0:top_k]
        
        y_pred = model_RF.predict(X_test)
        validation_rmse = mean_squared_error(y_test, y_pred, squared=False)
        details['validation_rmse'] = validation_rmse
        
        df = df.append(details, ignore_index=True)
        filepath = Path(dataset + '_RF_default.csv')
        filepath.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(filepath, index=False)

Facebook_data


 10%|███████████████████▌                                                                                                                                                                                | 1/10 [00:00<00:02,  4.01it/s]

Training RMSE:  73.67027507301252 Testing RMSE:  74.26660217531611


 20%|███████████████████████████████████████▏                                                                                                                                                            | 2/10 [00:00<00:01,  4.23it/s]

Training RMSE:  18.748803517380335 Testing RMSE:  87.80472309999426


...

Training RMSE:  1876.7033435882875 Testing RMSE:  4196.649884746348


 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                   | 9/10 [00:02<00:00,  4.07it/s]

Training RMSE:  1740.0305093539703 Testing RMSE:  5061.243594432188


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.04it/s]

Training RMSE:  1813.1508921340844 Testing RMSE:  5058.398061998935
